#### Generating Traning Result CSV

1. Importing the libraries

In [1]:
import os
import glob
import pickle
import sys
import platform

#appending path to lsr tensor 

if platform.system() == "Windows":
    sys.path.append(rf'D:\Tensor Based ML for Neuro Imaging\INSPIRE_CAHBHIR\Python Scripts\LSR-Tensor-Ridge-Regression\Closed_Form_Solver\Code Files')
    sys.path.append(rf'D:\Tensor Based ML for Neuro Imaging\INSPIRE_CAHBHIR\Python Scripts\LSR-Tensor-Ridge-Regression\Closed_Form_Solver\Experimental Results\Tensor_Method\Closed Form')
elif platform.system() == "Darwin":
    sys.path.append('/Users/lakrama/Neuro Project Codes/LSR-Tensor-Ridge-Regression/Closed_Form_Solver/Code Files')

import LSR_Tensor_2D_v1
from optimization import inner_product, R2, objective_function_vectorized




2. Generating CSV

In [ ]:
# The dataframe to hold the results for a seed

columns = ['Seed', 'Best Lambda', 'NMSE', 'CORR', 'R2']
results_df = pd.DataFrame(columns=columns)


# Specify the directory containing the files
directory = rf"D:\Tensor Based ML for Neuro Imaging\INSPIRE_CAHBHIR\Python Scripts\LSR-Tensor-Ridge-Regression_All_Data\Closed_Form_Solver\LSR\Separation Rank 2\Normalize_Int_Removal\Frobenious\679"

# Define the pattern to match the files
pattern = "*.pkl"

# Get the list of files matching the pattern
file_list = glob.glob(os.path.join(directory, pattern))

for file in file_list:
    with open(file, 'rb') as f:
        
        #loading data
        data = pickle.load(f)
        
        #extracting the final matrices
        X_train = data[0]
        Y_train = data[1]
        lambda1 = data[4]
        lsr_ten = data[18][-1]

        #generating the lsr tensor 
        Matrix = lsr_ten.expand_to_tensor()

        #training predictions
        Y_train_predicted = inner_product(np.transpose(X_train, (0, 2, 1)), Matrix.flatten(order ='F')) 

        train_nmse_loss = np.sum(np.square((Y_train_predicted.flatten() - Y_train.flatten()))) / np.sum(np.square(Y_train.flatten()))    
        train_R2_loss = R2(Y_train.flatten(), Y_train_predicted.flatten())
        train_correlation = np.corrcoef(Y_train_predicted.flatten(), Y_train.flatten())[0, 1]
        
        # Append the results to the dataframe
        seed_result_df = pd.DataFrame([{
            'Seed': file,
            'Best Lambda': lambda1,
            'NMSE': train_nmse_loss,
            'CORR': train_correlation,
            'R2': train_R2_loss,
        }])
        
        #concatenating the results
        results_df = pd.concat([results_df,seed_result_df], ignore_index = True) 
        
        
    # Define platform-specific file paths
    if platform.system() == 'Windows':
        file_path = r'D:\Tensor Based ML for Neuro Imaging\INSPIRE_CAHBHIR\Python Scripts\LSR-Tensor-Ridge-Regression_All_Data\Closed_Form_Solver\LSR\Separation Rank 2\Normalize_Int_Removal\Frobenious\679\Results'
    elif platform.system() == 'Darwin':
        file_path = 'addpath'  # Replace 'addpath' with the actual path for macOS

    # Save the result as a CSV
    results_df.to_csv(f'{file_path}/train_results_with_matrix.csv', index=False)
